# Chargement des données

In [14]:
# code inspiré de https://git.regardscitoyens.org/regardscitoyens/sprint-scrutins/blob/master/votes_an/ancsv.py

import glob, json, csv

# recupere les députés

députés = list(csv.DictReader(open('liste_deputes_libre_office.csv')))
députés_par_id = {int(député["identifiant"]): député for député in députés}

# recuperer les votes

votes_par_scrutins = {}

for fichier in glob.glob('scrutins/*.json'):
    scrutin = json.load(open(fichier))['scrutin']

    id_scrutin = scrutin['numero']
    votes_par_scrutins[id_scrutin] = {}

    if int(scrutin['syntheseVote']['nombreVotants']) < 50:
        continue
        
    groupes = scrutin['ventilationVotes']['organe']['groupes']['groupe']

    for groupe in groupes:
        dn = groupe['vote']['decompteNominatif']
        
        for position in ('pour', 'contre', 'abstention', 'nonVotant'):
            votants_position = dn['%ss' % position]
            
            if not votants_position:
                continue
            
            votants = votants_position['votant']

            if not isinstance(votants, list):
                votants = [votants]

            for votant in votants:
                votes_par_scrutins[id_scrutin][int(votant['acteurRef'][2:])] = position



def similarité(député1, député2):
    global votes
    total = 0
    total_en_commun = 0
    for scrutin, vote in votes_par_scrutins.items():
        if not vote.get(député1) or not vote.get(député2):
            continue
        if vote.get(député1) == vote.get(député2):
            total_en_commun += 1
        total += 1
    if total == 0:
        return 1
    return 1 - total_en_commun / total

print('sim mel/ruffin:', similarité(2150, 722142))
print('sim bergé/ruffin:', similarité(722046, 722142))

sim mel/ruffin: 0.014388489208633115
sim bergé/ruffin: 0.9144736842105263


# Carte

In [15]:
import umap
import umap.plot

def metric(a, b):
    return similarité(int(a[0]), int(b[0]))

députés_id = [int(x) for x in list(sorted(députés_par_id.keys()))]
partis = [députés_par_id[id]['Groupe politique (complet)'] for id in députés_id]
noms = [(députés_par_id[id]['Nom'] + ' ' + députés_par_id[id]['Prénom']) for id in députés_id]

mapper = umap.UMAP(metric=metric, random_state=30).fit([[id] for id in députés_id])

umap.plot.output_notebook()
p = umap.plot.interactive(mapper, labels=partis, hover_data={"parti": partis, "nom": noms})
umap.plot.show(p)

/home/damien/.venv/lib/python3.6/site-packages/umap/umap_.py:1495: UserWarning: custom distance metric does not return gradient; inverse_transform will be unavailable. To enable using inverse_transform method method, define a distance function that returns a tuple of (distance [float], gradient [np.array])
  "custom distance metric does not return gradient; inverse_transform will be unavailable. "


Loading BokehJS ...

# Députés les plus similaires à François Ruffin

In [16]:
sims = [[id, similarité(id, 722142)] for id in députés_id]
sims.sort(key=lambda x: x[1])
for sim in sims:
    print((députés_par_id[sim[0]]['Nom'] + ' ' + députés_par_id[sim[0]]['Prénom']), sim[1])

Rouaux Claudia 0.0
Corbière Alexis 0.0
Ruffin François 0.0
Quatennens Adrien 0.005494505494505475
Coquerel Éric 0.0055248618784530246
Lachaud Bastien 0.005649717514124242
Rubin Sabine 0.00666666666666671
Taurine Bénédicte 0.007575757575757569
Autain Clémentine 0.008547008547008517
Panot Mathilde 0.011428571428571455
Obono Danièle 0.012195121951219523
Mélenchon Jean-Luc 0.014388489208633115
Fiat Caroline 0.016129032258064502
Ressiguier Muriel 0.016260162601625994
Ratenon Jean-Hugues 0.01948051948051943
Prud'homme Loïc 0.020979020979020935
Bernalicis Ugo 0.021276595744680882
Larive Michel 0.021897810218978075
Jumel Sébastien 0.05464480874316935
Dharréville Pierre 0.05769230769230771
Faucillon Elsa 0.05932203389830504
Lecoq Jean-Paul 0.06134969325153372
Peu Stéphane 0.06432748538011701
Bruneel Alain 0.0650887573964497
Buffet Marie-George 0.06976744186046513
Bello Huguette 0.07058823529411762
Chassaigne André 0.07262569832402233
Roussel Fabien 0.0964912280701754
Dufrègne Jean-Paul 0.098837